In [1]:
!pip install transformers torch streamlit google-cloud-storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.

In [2]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [3]:
import pandas as pd

data = pd.read_csv('/content/ScrapedData_with_Sentiment.csv')
data

,Company,Title,sentiment,positive,negative,neutral
0,Absa Bank Kenya Plc,Absa Bank Kenya Plc Profit up 14% On Strong Ba...,positive,0.951615,0.023957,0.024429
1,Absa Bank Kenya Plc,Absa Bank Kenya PLC commits Ksh100 billion to ...,neutral,0.332196,0.007787,0.660017
2,Absa Bank Kenya Plc,Absa Bank Kenya PLC records 161% Rise in Profi...,positive,0.948649,0.020518,0.030833
3,Absa Bank Kenya Plc,Absa Bank Kenya Plc Media Statement,neutral,0.043061,0.041324,0.915614
4,Absa Bank Kenya Plc,"HSBC to Exit South Africa in Deals with Absa, ...",neutral,0.178253,0.008735,0.813011
...,...,...,...,...,...,...
2490,Unga Group Ltd,April Monthly Report,neutral,0.028002,0.071664,0.900334
2491,Unga Group Ltd,Q1 2016 Market Analysis: ‘Back To Where We Began’,neutral,0.064372,0.024908,0.910720
2492,Unga Group Ltd,Kakuzi Limited Analysis,neutral,0.034653,0.029309,0.936038
2493,Unga Group Ltd,Centum Investment Company Analysis,neutral,0.029194,0.024229,0.946577


In [4]:
texts = data['Title'].tolist()
labels = data['sentiment'].tolist()

In [7]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
import pandas as pd

# Load your data

texts = data['Title'].tolist()
labels = data['sentiment'].tolist()

# Create a label mapping
label_mapping = {
    'positive': 2,
    'neutral': 1,
    'negative': 0
}

# Convert string labels to integers
labels = [label_mapping[label] for label in labels]

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')

# Tokenize the texts
max_length = 512
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=max_length)

# Convert labels to a tensor
labels = torch.tensor(labels)

# Create a dataset
from torch.utils.data import Dataset, DataLoader

class StockNewsDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.inputs.items()}
        item['labels'] = self.labels[idx]
        return item

dataset = StockNewsDataset(inputs, labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Create a Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mutwiri333 (mutwiri333-pipsy-hussle) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
500,1.116600


TrainOutput(global_step=936, training_loss=0.6701961093478732, metrics={'train_runtime': 3964.5508, 'train_samples_per_second': 1.888, 'train_steps_per_second': 0.236, 'total_flos': 123087745730880.0, 'train_loss': 0.6701961093478732, 'epoch': 3.0})

In [8]:
model.save_pretrained("finbert_finetuned")
tokenizer.save_pretrained("finbert_finetuned")


('finbert_finetuned/tokenizer_config.json',
 'finbert_finetuned/special_tokens_map.json',
 'finbert_finetuned/vocab.txt',
 'finbert_finetuned/added_tokens.json')

In [9]:
!zip -r finbert_finetuned.zip finbert_finetuned


  adding: finbert_finetuned/ (stored 0%)
  adding: finbert_finetuned/special_tokens_map.json (deflated 42%)
  adding: finbert_finetuned/config.json (deflated 50%)
  adding: finbert_finetuned/vocab.txt (deflated 50%)
  adding: finbert_finetuned/tokenizer_config.json (deflated 74%)
  adding: finbert_finetuned/model.safetensors (deflated 7%)


In [10]:
from google.colab import files
files.download("finbert_finetuned.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>